#Group members:
-  Bao Luu
-  Lev Nguyen
-  Sean Safi

#Central Question: 
**What are the major factors contributing to the huge number of casualties of the Russia-Ukraine war**

#Data sources

-  Russian losses in Russia-Ukraine war(JSON): https://drive.google.com/drive/folders/1nfhloimOLZ06Qe5KwPbVcggNtWJ0c8gM?usp=share_link ([Source](https://www.kaggle.com/datasets/299939cceb4d5b67df96794141a0e2ee74fd0f054ab61f72afbfbde27a0ae8e7?resource=download&select=russo-ukraine-war-casualties.json) )
-  Wikipedia website: https://en.wikipedia.org/wiki/Casualties_of_the_Russo-Ukrainian_War#2022_Russian_invasion_of_Ukraine

#Deliverable 2

##Importing libraries

In [171]:
import json
import pandas as pd
import numpy as np
import requests
from lxml import etree
import io
import sqlite3 as sql

##Processing the JSON Dataset

Create a function `read_data()` taking the name of JSON file as the only argument, returning its data as a list of dictionaries

In [172]:
def read_data(file_name):
  with open(f'{file_name}', 'r') as fi:
    data = json.load(fi)
  return data

In [173]:
total_casualties = read_data('russo-ukraine-war-casualties.json')
print(total_casualties)

[{'date': 1645660800000, 'tanks': 30, 'armored_vehicle': 130, 'planes': 7, 'helicopters': 6, 'cannons': 0, 'mlrs_buk': 0, 'mlrs_grad': 0, 'mlrs': 0, 'anti_air': 0, 'uav': 0, 'cruise_missiles': 0, 'ships': 0, 'cars_cisterns': 0, 'special_equpment': 0, 'personnel': 800}, {'date': 1645747200000, 'tanks': 100, 'armored_vehicle': 516, 'planes': 10, 'helicopters': 7, 'cannons': 0, 'mlrs_buk': 0, 'mlrs_grad': 0, 'mlrs': 0, 'anti_air': 0, 'uav': 0, 'cruise_missiles': 0, 'ships': 0, 'cars_cisterns': 0, 'special_equpment': 0, 'personnel': 2800}, {'date': 1645833600000, 'tanks': 100, 'armored_vehicle': 540, 'planes': 16, 'helicopters': 18, 'cannons': 0, 'mlrs_buk': 0, 'mlrs_grad': 0, 'mlrs': 0, 'anti_air': 0, 'uav': 0, 'cruise_missiles': 0, 'ships': 0, 'cars_cisterns': 0, 'special_equpment': 0, 'personnel': 3000}, {'date': 1645920000000, 'tanks': 150, 'armored_vehicle': 706, 'planes': 27, 'helicopters': 26, 'cannons': 50, 'mlrs_buk': 1, 'mlrs_grad': 4, 'mlrs': 0, 'anti_air': 0, 'uav': 2, 'cruise_

From the JSON file data, we can see that the dates are in epoch/Unix time - the number of seconds that have elapsed since 00:00:00 UTC on 1 January 1970, the beginning of the Unix epoch. That being said, we have to convert the epoch times to actual dates and times using the `datetime` package

First, we create a function `date_converter()` that takes in our list of epoch times and return the list of actual dates. The strftime() function converts the datetime objects into datetime strings, and its parameters are the format codes in which we wanted to customize the string of date 

In [174]:
import datetime

def date_converter(epoch_time):
  date_time = datetime.datetime.fromtimestamp(epoch_time) #datetime.fromtimestamp() converts the epoch time into DateTime
  dates = date_time.strftime( "%Y - %m - %d  %H : %M : %S")

  return dates

Loop through the LoD `total_casualties`, assigning the list of all values of the `date` keys in the dictionaries to variable `epoch_time`. Pass the list of epoch times in the `date_converter()` function and store all outputs in `converted` variable. Finally, assign all the `date` keys the converted values

In [175]:
for i in total_casualties:
  epoch_time = i['date']
  #print(epoch_time)
  epoch_time = epoch_time // 1000
  converted = date_converter(epoch_time)
  i['date'] = converted
  
total_casualties

[{'date': '2022 - 02 - 24  00 : 00 : 00',
  'tanks': 30,
  'armored_vehicle': 130,
  'planes': 7,
  'helicopters': 6,
  'cannons': 0,
  'mlrs_buk': 0,
  'mlrs_grad': 0,
  'mlrs': 0,
  'anti_air': 0,
  'uav': 0,
  'cruise_missiles': 0,
  'ships': 0,
  'cars_cisterns': 0,
  'special_equpment': 0,
  'personnel': 800},
 {'date': '2022 - 02 - 25  00 : 00 : 00',
  'tanks': 100,
  'armored_vehicle': 516,
  'planes': 10,
  'helicopters': 7,
  'cannons': 0,
  'mlrs_buk': 0,
  'mlrs_grad': 0,
  'mlrs': 0,
  'anti_air': 0,
  'uav': 0,
  'cruise_missiles': 0,
  'ships': 0,
  'cars_cisterns': 0,
  'special_equpment': 0,
  'personnel': 2800},
 {'date': '2022 - 02 - 26  00 : 00 : 00',
  'tanks': 100,
  'armored_vehicle': 540,
  'planes': 16,
  'helicopters': 18,
  'cannons': 0,
  'mlrs_buk': 0,
  'mlrs_grad': 0,
  'mlrs': 0,
  'anti_air': 0,
  'uav': 0,
  'cruise_missiles': 0,
  'ships': 0,
  'cars_cisterns': 0,
  'special_equpment': 0,
  'personnel': 3000},
 {'date': '2022 - 02 - 27  00 : 00 : 00',


##Web scraping 


###Function Preparation

1. Function `process_data()` converts the passed data into a pandas DataFrame

2. Function `clean_data()` elimninates the unwanted '\n' from all the data scraped from the web   

3. Function `html_parser()` parses all the data of the web page that we need to scrape, returning the root element of the tree of data parsed

In [176]:
def process_data(data):
  df = pd.DataFrame(data)
  return df

In [177]:
def clean_data(raw_data):
  L = list() #L is new list storing our cleaned data 
  for i in raw_data:
    L.append(i.strip('\n'))
  return L

In [178]:
def html_parser(url):
  response = requests.get(url)

  assert response.status_code == 200

  htmlparser = etree.HTMLParser()
  tree = etree.parse(io.BytesIO(response.content), htmlparser)
  root = tree.getroot()
  return root

###**Web Scraping Table 1: Civilian Deaths**

To get the table 'Civilian Deaths' from the Wikipedia page, we need to parse the data first using the `html_parser()` function, assigning the root element of the web page's tree of elements to variable `root1`. Then, we can use xpath expression to extract the table we need.

In [179]:
root1 = html_parser('https://en.wikipedia.org/wiki/Casualties_of_the_Russo-Ukrainian_War')
node_table1 = root1.xpath('//*[@id="mw-content-text"]/div[1]/table[4]')
print(node_table1)

[<Element table at 0x7f9ad495f600>]


After retrieving the table, we can reuse the xpath expression to get different values inside the table

In [180]:
cols = node_table1[0].xpath('./tbody/tr/td[1]/table/tbody/tr[1]/th/text()')
areas = node_table1[0].xpath('./tbody/tr/td[1]/table/tbody/tr[*]/th[*]/b/a/text()')
fatalities = node_table1[0].xpath('./tbody/tr/td[1]/table/tbody/tr[*]/td[*]/b/text()')
time_period = node_table1[0].xpath('./tbody/tr/td[1]/table/tbody/tr[*]/td[2]/text()')
source = node_table1[0].xpath('./tbody/tr/td[1]/table/tbody/tr[2]/th[2]/text()')
print(source, cols, areas, fatalities, time_period)

['Ukrainian government\n'] ['Area\n', 'Fatalities\n', 'Time period\n', 'Source\n'] ['Cherkasy Oblast', 'Chernihiv Oblast', 'Dnipropetrovsk Oblast', 'Donetsk Oblast', 'Kharkiv Oblast', 'Kherson Oblast', 'Kirovohrad Oblast', 'Kyiv Oblast', 'Luhansk Oblast', 'Lviv Oblast', 'Mariupol', 'Mykolaiv Oblast', 'Odesa Oblast', 'Poltava Oblast', 'Rivne Oblast', 'Sumy Oblast', 'Vinnytsia Oblast', 'Volyn Oblast', 'Zaporizhzhia Oblast', 'Zhytomyr Oblast'] ['2', '700+', '53', '1,246', '1,600+', '467', '7', '1,596+', '1,986+', '7', '25,000+', '403', '33', '22', '25', '106+', '23', '5', '113', '13', '33,407+'] ['24 February – 26 June 2022\n', '24 February – 29 March 2022\n', '24 April – 25 October 2022\n', '24 February – 8 December 2022\n', '24 February – 7 December 2022\n', '24 February – 24 November 2022\n', '24 February – 28 July 2022\n', '24 February – 11 October 2022\n', '24 February – 1 October 2022\n', '18 April 2022\n', '24 February – 25 May 2022\n', '24 February – 2 August 2022\n', '24 February

From observation, the extracted values from the table has some unwanted characters `\n`, so we can use the function `clean_data()` to eliminate those characters from the table values

In [181]:
cols = clean_data(cols)
time_period = clean_data(time_period)
source = clean_data(source)
print(cols, time_period, source)

['Area', 'Fatalities', 'Time period', 'Source'] ['24 February – 26 June 2022', '24 February – 29 March 2022', '24 April – 25 October 2022', '24 February – 8 December 2022', '24 February – 7 December 2022', '24 February – 24 November 2022', '24 February – 28 July 2022', '24 February – 11 October 2022', '24 February – 1 October 2022', '18 April 2022', '24 February – 25 May 2022', '24 February – 2 August 2022', '24 February – 23 September 2022', '27 June 2022', '24 February – 23 June 2022', '24 February – 18 October 2022', '14 July 2022', '24 February – 25 July 2022', '24 February – 11 December 2022', '24 February – 10 March 2022', '24 February – 11 December 2022'] ['Ukrainian government']


After cleaning all the data, we can create a List of Lists representation of the table from the data extracted above by looping through all the indexes of each list (areas, fatalities, time_period, source), append the corresponding values from each list into the list `temp`, and append it to our LoL

In [182]:
LoL1 = list()
for value in range(len(areas)):
  temp = []
  temp.append(areas[value])
  temp.append(fatalities[value])
  temp.append(time_period[value])
  temp.append(source[0])
  LoL1.append(temp)
LoL1

[['Cherkasy Oblast',
  '2',
  '24 February – 26 June 2022',
  'Ukrainian government'],
 ['Chernihiv Oblast',
  '700+',
  '24 February – 29 March 2022',
  'Ukrainian government'],
 ['Dnipropetrovsk Oblast',
  '53',
  '24 April – 25 October 2022',
  'Ukrainian government'],
 ['Donetsk Oblast',
  '1,246',
  '24 February – 8 December 2022',
  'Ukrainian government'],
 ['Kharkiv Oblast',
  '1,600+',
  '24 February – 7 December 2022',
  'Ukrainian government'],
 ['Kherson Oblast',
  '467',
  '24 February – 24 November 2022',
  'Ukrainian government'],
 ['Kirovohrad Oblast',
  '7',
  '24 February – 28 July 2022',
  'Ukrainian government'],
 ['Kyiv Oblast',
  '1,596+',
  '24 February – 11 October 2022',
  'Ukrainian government'],
 ['Luhansk Oblast',
  '1,986+',
  '24 February – 1 October 2022',
  'Ukrainian government'],
 ['Lviv Oblast', '7', '18 April 2022', 'Ukrainian government'],
 ['Mariupol', '25,000+', '24 February – 25 May 2022', 'Ukrainian government'],
 ['Mykolaiv Oblast',
  '403',
  

###**Web Scraping Table 2: Foreign fighters and volunteers (Vy's)**

In [183]:
node_table2 = root1.xpath('//*[@id="mw-content-text"]/div[1]/table[6]')
#print(node_table2)

node_table3 = root1.xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[2]')
#print(node_table3)

#### Dead foreign fighters of the 2022 Russian invasion of Ukraine 
In Australia, Azerbaijan, Belarus, France and Georgia, there are multiple allegiances. However, in the wikipage, these allegiances are separated by "\n". Thus, after cleaning up the data, we cannot fit all those allegiances in their corresponding rows. Our solution is to manually process the data from this column so as to match the head rows.

In [184]:
dead_country = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[1]/table/tbody/tr[*]/td[1]/a/text()')
deaths = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[1]/table/tbody/tr[*]/td[2]/text()')
deaths = [x.strip("\n") for x in deaths]
dead_allegiance = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[1]/table/tbody/tr[*]/td[3]/a/text()')
dead_allegiance[1] = dead_allegiance[1] + " and " + dead_allegiance[2]
dead_allegiance.remove(dead_allegiance[2])
dead_allegiance[3] = dead_allegiance[3] + " and " + dead_allegiance[4]
dead_allegiance.remove(dead_allegiance[4])
dead_allegiance[4] = dead_allegiance[4] + " and " + dead_allegiance[5]
dead_allegiance.remove(dead_allegiance[5])
dead_allegiance[11] = dead_allegiance[11] + " and " + dead_allegiance[12]
dead_allegiance.remove(dead_allegiance[12])
dead_allegiance[12] = dead_allegiance[12] + ", " + dead_allegiance[13] + ", " + dead_allegiance[14] + ", " + dead_allegiance[15] + ", " + dead_allegiance[16] + " and " + dead_allegiance[17]
for i in range(5):
  dead_allegiance.remove(dead_allegiance[13])

count = 0
death_forces = []
for val in deaths:
  count += int(val)
  if count < 137:
    death_forces.append('Ukrainian Armed forces')
  elif count < (137 + 46):
    death_forces.append('Russian Armed forces')
  elif count < (133 + 46 + 6):
    death_forces.append('Donetsk Armed forces')
  else:
    death_forces.append('Luhansk Armed forces')
print(dead_allegiance)

['Ukrainian Foreign Legion', 'Ukrainian Foreign Legion and Sich Battalion', 'Ukrainian Armed Forces', 'Armed Forces of Ukraine and Georgian Legion', 'Kastuś Kalinoŭski Battalion and 24th Mechanized Brigade', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Armed Forces of Ukraine', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion and Sich Battalion', 'Georgian Legion, Sich Battalion, Azov Battalion, Dzhokhar Dudayev Battalion, 25th TDB "Kyivska Rus" and 57th Motorized Brigade', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Sich Battalion', 'Ukrainian Foreign Legion', 'Ukrainian Foreign Legion', 'Private military company', 'Russian Army'

In [185]:
LoL2 = list()
cols2 =['Country','Deaths','Allegiance','Forces']
for value in range(len(dead_country)):
  temp = []
  temp.append(dead_country[value])
  temp.append(deaths[value])
  temp.append(dead_allegiance[value])
  temp.append(death_forces[value])
  LoL2.append(temp)

LoL2

[['Argentina', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['Australia',
  '3',
  'Ukrainian Foreign Legion and Sich Battalion',
  'Ukrainian Armed forces'],
 ['Austria', '1', 'Ukrainian Armed Forces', 'Ukrainian Armed forces'],
 ['Azerbaijan',
  '25',
  'Armed Forces of Ukraine and Georgian Legion',
  'Ukrainian Armed forces'],
 ['Belarus',
  '16',
  'Kastuś Kalinoŭski Battalion and 24th Mechanized Brigade',
  'Ukrainian Armed forces'],
 ['Brazil', '3', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['Canada', '2', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['Colombia', '3', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['Croatia', '1', 'Armed Forces of Ukraine', 'Ukrainian Armed forces'],
 ['Czech Republic', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['Denmark', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'],
 ['France',
  '3',
  'Ukrainian Foreign Legion and Sich Battalion',
  'Ukrainian Armed forces'],
 

#### Captured foreign fighters of the 2022 Russian invasion of Ukraine 

In [186]:
captured_country = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[*]/td[1]/a/text()')
captured = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[*]/td[2]/text()')
captured = [x.strip("\n") for x in captured]
captured_allegiance = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[*]/td[3]/a/text()')
status = node_table2[0].xpath('//*[@id="mw-content-text"]/div[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[*]/td[4]/text()')
status = [x.strip("\n") for x in status]
status[-1] = status[-2] +" "+ status.pop()
captured_forces = ['Ukrainian Armed Forces' for x in captured_country]

In [187]:
LoL3 = list()
cols3 =['Country','Captured','Allegiance','Forces']
for value in range(len(captured_country)):
  temp = []
  temp.append(captured_country[value])
  temp.append(captured[value])
  temp.append(captured_allegiance[value])
  temp.append(captured_forces[value])
  LoL3.append(temp)
LoL3

[['Belarus', '2', 'Kastuś Kalinoŭski Battalion', 'Ukrainian Armed Forces'],
 ['Croatia', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces'],
 ['Israel', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces'],
 ['Morocco', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces'],
 ['Sweden', '1', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces'],
 ['United Kingdom', '4', 'Armed Forces of Ukraine', 'Ukrainian Armed Forces'],
 ['United States', '3', 'Ukrainian Foreign Legion', 'Ukrainian Armed Forces']]

#Deliverable 3

##Creating SQL Database

Now we have to create a SQL database `ukraine_war.db`, storing 3 tables:
-  Civilian Deaths by Regions (Table 1)
-  Dead foreign fighters (Table 2)
-  Total Casualties (JSON File data)


###Preparing functions
1.  `sql_create(db, qry)` takes 2 arguments: database and the SQL query, returning the database

2.  `sql_select(db, qry)` takes 2 arguments database and SQL query, returning a subset of the data from the SQL Database created

3. `sql_insert_many(db, qry, data)` populate the tables in the database with the data parsed from the data sources

In [188]:
def sql_create(db, cqry):
  connection = sql.connect(db)
  cursor = connection.cursor()
  cursor.execute(cqry)

In [189]:
def sql_select(db, ssqry):
  connection = sql.connect(db)
  cursor = connection.cursor()
  subset = cursor.execute(ssqry)

  return subset.fetchall()

In [190]:
def sql_insert_many(db, iqry, data):
  conn = sql.connect(db)
  cur = conn.cursor()
  cur.executemany(iqry, data)
  conn.commit()

###**Table 1: Civilian Deaths**

-  For this table, we are inserting `LoL2` - The table Civilian Deaths by Area that we scraped from the Wikipedia page.

In [191]:
db = 'ukraine_war.db'

cqry = ''' 
CREATE TABLE Civilian_deaths (
  Area VARCHAR(255) NOT NULL,
  Fatalities INT NOT NULL,
  Time Period TEXT,
  Source VARCHAR(255)
  ); 
'''

sql_create(db, cqry)

In [192]:
iqry = 'INSERT INTO Civilian_deaths VALUES(?, ?, ?,?)'

sql_insert_many(db,iqry,LoL1)

In [193]:
sqry = 'SELECT * from Civilian_deaths'
sql_select(db,sqry)

[('Cherkasy Oblast', 2, '24 February – 26 June 2022', 'Ukrainian government'),
 ('Chernihiv Oblast',
  '700+',
  '24 February – 29 March 2022',
  'Ukrainian government'),
 ('Dnipropetrovsk Oblast',
  53,
  '24 April – 25 October 2022',
  'Ukrainian government'),
 ('Donetsk Oblast',
  '1,246',
  '24 February – 8 December 2022',
  'Ukrainian government'),
 ('Kharkiv Oblast',
  '1,600+',
  '24 February – 7 December 2022',
  'Ukrainian government'),
 ('Kherson Oblast',
  467,
  '24 February – 24 November 2022',
  'Ukrainian government'),
 ('Kirovohrad Oblast',
  7,
  '24 February – 28 July 2022',
  'Ukrainian government'),
 ('Kyiv Oblast',
  '1,596+',
  '24 February – 11 October 2022',
  'Ukrainian government'),
 ('Luhansk Oblast',
  '1,986+',
  '24 February – 1 October 2022',
  'Ukrainian government'),
 ('Lviv Oblast', 7, '18 April 2022', 'Ukrainian government'),
 ('Mariupol', '25,000+', '24 February – 25 May 2022', 'Ukrainian government'),
 ('Mykolaiv Oblast',
  403,
  '24 February – 2 A

###**Table 2: Dead Foreign Fighters**

In [194]:
db = 'ukraine_war.db'

cqry = '''
CREATE TABLE Dead_foreign_fighters (
  Country VARCHAR(200) NOT NULL,
  Deaths INT NOT NULL,
  Allegiance VARCHAR(100) NOT NULL,
  Forces VARCHAR(200)
);
'''
sql_create(db, cqry)

In [195]:
iqry = 'INSERT INTO Dead_foreign_fighters VALUES(?, ?, ?, ?)'
data = LoL2

sql_insert_many(db, iqry, data)

In [196]:
sqry = 'SELECT * FROM Dead_foreign_fighters'
sql_select(db, sqry)

[('Argentina', 1, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('Australia',
  3,
  'Ukrainian Foreign Legion and Sich Battalion',
  'Ukrainian Armed forces'),
 ('Austria', 1, 'Ukrainian Armed Forces', 'Ukrainian Armed forces'),
 ('Azerbaijan',
  25,
  'Armed Forces of Ukraine and Georgian Legion',
  'Ukrainian Armed forces'),
 ('Belarus',
  16,
  'Kastuś Kalinoŭski Battalion and 24th Mechanized Brigade',
  'Ukrainian Armed forces'),
 ('Brazil', 3, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('Canada', 2, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('Colombia', 3, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('Croatia', 1, 'Armed Forces of Ukraine', 'Ukrainian Armed forces'),
 ('Czech Republic', 1, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('Denmark', 1, 'Ukrainian Foreign Legion', 'Ukrainian Armed forces'),
 ('France',
  3,
  'Ukrainian Foreign Legion and Sich Battalion',
  'Ukrainian Armed forces'),
 ('Georgia',
  35,
  'Geo

###**Table 3: Total Casualties**

-  Since the data read from the JSON file is currently a List of Dictionaries, we have to convert it to a List of Lists representation so we can populate the table total_casualties in our Database.

-  The code creates two lists: columns, which contains the keys from each dictionary in total_casualties, and values, which contains the values from each dictionary in total_casualties in the same order as the keys in columns.

In [197]:
columns = []

for data in total_casualties:
  for col in data.keys():
    if col not in columns:
      columns.append(col)

values = []

for data in total_casualties:
  value = []
  for col in columns:
    value.append(str(data.get(col)))
  values.append(value)

After converting the LoD to LoL, we can insert the data into the table

In [198]:
db = 'ukraine_war.db'

cqry = '''
CREATE TABLE total_losses (
  date VARCHAR(200) NOT NULL,
  tanks INT NOT NULL,
  armored_vehicle INT NOT NULL,
  planes INT NOT NULL,
  helicopters INT NOT NULL,
  cannons INT NOT NULL,
  mlrs_buk INT NOT NULL,
  mlrs_grad INT NOT NULL,
  mlrs INT NOT NULL,
  anti_air INT NOT NULL,
  uav INT NOT NULL,
  cruise_missiles INT NOT NULL,
  ships INT NOT NULL,
  cars_cisterns INT NOT NULL,
  special_equipment INT NOT NULL,
  personnel INT NOT NULL
);
'''
sql_create(db, cqry)

iqry = 'INSERT INTO total_losses VALUES(?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?)'


sql_insert_many(db, iqry, values)

sqry = 'SELECT * FROM total_losses'
sql_select(db, sqry)

[('2022 - 02 - 24  00 : 00 : 00',
  30,
  130,
  7,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  800),
 ('2022 - 02 - 25  00 : 00 : 00',
  100,
  516,
  10,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2800),
 ('2022 - 02 - 26  00 : 00 : 00',
  100,
  540,
  16,
  18,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3000),
 ('2022 - 02 - 27  00 : 00 : 00',
  150,
  706,
  27,
  26,
  50,
  1,
  4,
  0,
  0,
  2,
  0,
  2,
  0,
  0,
  4500),
 ('2022 - 02 - 28  00 : 00 : 00',
  191,
  816,
  29,
  29,
  74,
  1,
  21,
  0,
  0,
  3,
  0,
  2,
  0,
  0,
  5300),
 ('2022 - 03 - 01  00 : 00 : 00',
  198,
  846,
  29,
  29,
  77,
  1,
  24,
  0,
  7,
  3,
  0,
  2,
  0,
  0,
  5710),
 ('2022 - 03 - 02  00 : 00 : 00',
  211,
  862,
  30,
  31,
  85,
  0,
  40,
  0,
  9,
  3,
  0,
  2,
  0,
  0,
  5840),
 ('2022 - 03 - 03  00 : 00 : 00',
  217,
  900,
  30,
  31,
  90,
  0,
  0,
  42,
  11,
  3,
  0,
  2,
  0,
  0,
  9000),
 ('2022 - 03 - 04  00 : 00 : 00',
  251,
  